# CoexistAI Tool Tutorial

Welcome to the tutorial for the coexistAI tool! This notebook will guide you through the main functionalities of the tool, including web search, document processing, generative models, answer generation, YouTube summarization, and more. Each section contains explanations and code examples to help you get started quickly.

## 1. Setup and Initialization

First, let's import the required libraries, set up environment variables, and initialize the main components. This ensures that all dependencies are loaded and ready for use.

In [ ]:
from utils.utils import *
set_logging(True) 
from langchain_text_splitters import TokenTextSplitter
import os

if not is_searxng_running():
       !docker run --rm \
                 -d -p 30:8080 \
                 -v "${PWD}/searxng:/etc/searxng" \
                 -e "BASE_URL=http://localhost:$PORT/" \
                 -e "INSTANCE_NAME=my-instance" \
                 searxng/searxng
else:
       print("SearxNG docker container is already running.")

os.environ['GOOGLE_API_KEY'] = 'YOUR_OWN_KEY'  # Replace with your actual key
text_splitter = TokenTextSplitter(chunk_size=512, chunk_overlap=128)
from utils.websearch_utils import *
searcher = SearchWeb(30)  # Initialize web search with a result limit

SearxNG docker container is already running.

## 2. Loading Models

Load embedding models and cross-encoders using the `load_model` function. You can choose between different embedding modes such as 'gemini', 'huggingface', or 'infinity_emb'.

In [10]:
hf_embeddings, cross_encoder = load_model("models/embedding-001", _embed_mode='gemini')

[2025-06-08 13:33:56,134] INFO utils.utils: Loading model: models/embedding-001 with embedding mode: gemini
[2025-06-08 13:33:56,134] INFO utils.utils: Loading model: models/embedding-001 with embedding mode: gemini
[2025-06-08 13:33:56,830] INFO transformers.configuration_utils: loading configuration file config.json from cache at /Users/sidhantthole/.cache/huggingface/hub/models--BAAI--bge-reranker-base/snapshots/2cfc18c9415c912f9d8155881c133215df768a70/config.json
[2025-06-08 13:33:56,830] INFO transformers.configuration_utils: loading configuration file config.json from cache at /Users/sidhantthole/.cache/huggingface/hub/models--BAAI--bge-reranker-base/snapshots/2cfc18c9415c912f9d8155881c133215df768a70/config.json
[2025-06-08 13:33:56,832] INFO transformers.configuration_utils: Model config XLMRobertaConfig {
  "architectures": [
    "XLMRobertaForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id":

## 4. Web Search Integration

Use the `SearchWeb` class to perform web searches and retrieve results. This is useful for augmenting LLMs with up-to-date information from the web.

In [11]:
results = searcher.query_search("latest news in AI", num_results=3)
print(results)

[2025-06-08 13:34:01,299] INFO utils.websearch_utils: Search results for query 'latest news in AI': [{'snippet': 'Anthropic launches Claude AI models for US national security · Reddit sues Anthropic over AI data scraping · Tackling hallucinations: MIT spinout teaches AI to ...', 'title': 'AI News | Latest AI News, Analysis & Events', 'link': 'https://www.artificialintelligence-news.com/', 'engines': ['google'], 'category': 'general'}, {'snippet': "Inside OpenAI's plan to embed ChatGPT into college students' lives · UK judge warns of risk to justice after lawyers cited fake AI-generated cases in court.", 'title': 'Artificial Intelligence: Read latest news updates on AI ...', 'link': 'https://indianexpress.com/section/technology/artificial-intelligence/', 'engines': ['google'], 'category': 'general'}, {'snippet': 'News coverage on artificial intelligence and machine learning tech, the companies building them, and the ethical issues AI raises today.', 'title': 'AI News & Artificial Intell

[
    {
        'snippet': 'Anthropic launches Claude AI models for US national security · Reddit sues Anthropic over AI 
data scraping · Tackling hallucinations: MIT spinout teaches AI to ...',
        'title': 'AI News | Latest AI News, Analysis & Events',
        'link': 'https://www.artificialintelligence-news.com/',
        'engines': ['google'],
        'category': 'general'
    },
    {
        'snippet': "Inside OpenAI's plan to embed ChatGPT into college students' lives · UK judge warns of risk to 
justice after lawyers cited fake AI-generated cases in court.",
        'title': 'Artificial Intelligence: Read latest news updates on AI ...',
        'link': 'https://indianexpress.com/section/technology/artificial-intelligence/',
        'engines': ['google'],
        'category': 'general'
    },
    {
        'snippet': 'News coverage on artificial intelligence and machine learning tech, the companies building 
them, and the ethical issues AI raises today.',
        'title': 'AI News & Artificial Intelligence',
        'link': 'https://techcrunch.com/category/artificial-intelligence/',
        'engines': ['google'],
        'category': 'general'
    }
]

## 5. Document Conversion from URLs

Convert URLs into document objects using the `urls_to_docs` function. This allows you to process and analyze web content as structured documents.

In [12]:
docs = await urls_to_docs([
    "https://en.wikipedia.org/wiki/India",
    "https://en.wikipedia.org/wiki/Bangalore"
])
print(f"Loaded {len(docs)} documents.")
print(docs[0])

/opt/homebrew/Cellar/python@3.13/3.13.1/Frameworks/Python.framework/Versions/3.13/lib/python3.13/multiprocessing/resource_tracker.py:136: UserWarning: resource_tracker: process died unexpectedly, relaunching.  Some resources might leak.
  warnings.warn('resource_tracker: process died unexpectedly, '
[2025-06-08 13:34:04,931] INFO utils.websearch_utils: Fetching URL: https://en.wikipedia.org/wiki/India


[2025-06-08 13:34:04,931] INFO utils.websearch_utils: Fetching URL: https://en.wikipedia.org/wiki/India
[2025-06-08 13:34:04,934] INFO utils.websearch_utils: Fetching URL: https://en.wikipedia.org/wiki/Bangalore
[2025-06-08 13:34:04,934] INFO utils.websearch_utils: Fetching URL: https://en.wikipedia.org/wiki/Bangalore
[2025-06-08 13:34:05,234] INFO utils.websearch_utils: Fetched content from https://en.wikipedia.org/wiki/India with type text/html; charset=UTF-8
[2025-06-08 13:34:05,234] INFO utils.websearch_utils: Fetched content from https://en.wikipedia.org/wiki/India with type text/html; charset=UTF-8
[2025-06-08 13:34:05,255] INFO utils.websearch_utils: Fetched content from https://en.wikipedia.org/wiki/Bangalore with type text/html; charset=UTF-8
[2025-06-08 13:34:05,255] INFO utils.websearch_utils: Fetched content from https://en.wikipedia.org/wiki/Bangalore with type text/html; charset=UTF-8
[2025-06-08 13:34:07,080] INFO utils.websearch_utils: Processed markdown for: https://en

Loaded 2 documents.

Document(
    metadata={'source': 'https://en.wikipedia.org/wiki/India'},
    page_content='\n\nIndia - WikipediaJump to contentCoordinates: 21Â°N 78Â°E\ufeff / \ufeff21Â°N 78Â°E\ufeff / 
21; 78\n![Featured \n![Extended-protected \n\nFrom Wikipedia, the free encyclopediaCountry in South Asia\nThis 
article is about the country. For other uses, see India (disambiguation).\n\n| Republic of India*BhÄ\x81rat 
Gaá¹\x87arÄ\x81jya* | |\n| --- | --- |\n| Horizontal tricolour flag bearing, from top to bottom, deep saffron, 
white, and green horizontal bands. In the centre of the white band is a navy-blue wheel with 24 spokes. Flag  State
emblem | |\n| **Motto:**Satyameva Jayate\xa0(Sanskrit)"Truth Alone Triumphs"[1] | |\n| **Anthem:**\xa0Jana Gana 
Mana\xa0 Art the Ruler of the Minds of All People"[4][2] | |\n| **National song: Vande Mataram\xa0(Sanskrit)[c]**"I
Bow to Thee, Mother"[b][1][2] | |\n| Image of a globe centred on India, with India highlighted.\xa0\xa0Territory 
controlled by India\xa0\xa0Territory claimed but not controlled | |\n| Capital | New Delhi28Â°36â\x80²50â\x80³N 
77Â°12â\x80²30â\x80³E\ufeff / \ufeff28.61389Â°N 77.20833Â°E\ufeff / 28.61389; 77.20833 |\n| Largest city by city 
proper population | Mumbai |\n| Largest city by metropolitan area population | Delhi |\n| Official\xa0languages |  
|\n| Recognised regional\xa0languages | State level and Eighth Schedule[9] |\n| Native languages | 424 languages[g]
|\n| Religion  (2011)[11] |  |\n| Demonym(s) |  |\n| Government | Federal parliamentary republic |\n|  | |\n| 
â\x80¢\xa0President | Droupadi Murmu |\n| â\x80¢\xa0Prime Minister | Narendra Modi |\n|  | |\n| Legislature | 
Parliament |\n| â\x80¢\xa0Upper house | Rajya Sabha |\n| â\x80¢\xa0Lower house | Lok Sabha |\n| 
Independence\xa0from the United Kingdom | |\n|  | |\n| â\x80¢\xa0Dominion | 15 August 1947 |\n| â\x80¢\xa0Republic 
| 26 January 1950 |\n|  | |\n| Area | |\n| â\x80¢\xa0Total | 3,287,263\xa0km2 (1,269,219\xa0sq\xa0mi)[2][h] (7th) 
|\n| â\x80¢\xa0Water\xa0(%) | 9.6 |\n| Population | |\n| â\x80¢\xa02023 estimate | Neutral increase 
1,428,627,663[13] (1st) |\n| â\x80¢\xa02011\xa0census | Neutral increase  (2nd) |\n| â\x80¢\xa0Density | 429.8/km2 
(1,113.2/sq\xa0mi) (30th) |\n| GDP\xa0(PPP) | 2025\xa0estimate |\n| â\x80¢\xa0Total | Increase $17.647 trillion[16]
(3rd) |\n| â\x80¢\xa0Per capita | Increase $12,132[16] (119th) |\n| GDP\xa0(nominal) | 2025\xa0estimate |\n| 
â\x80¢\xa0Total | Increase $4.187 trillion[16] (4th) |\n| â\x80¢\xa0Per capita | Increase $2,878[16] (136th) |\n| 
Gini\xa0(2021) | Positive decrease\xa032.8[17]medium inequality |\n| HDI\xa0(2023) | 
Increase\xa00.685[18]medium\xa0(130th) |\n| Currency | Indian rupee (â\x82¹) (INR) |\n| Time zone | UTC+05:30 (IST)
|\n| Date format |  |\n| Calling code | +91 |\n| ISO 3166 code | IN |\n| Internet TLD | .in (others) 
|\n\n**India**, officially the **Republic of India**,[j][20] is a country in South Asia. It is the seventh-largest 
country by area; the most populous country since 2023;[21] and, since its independence in 1947, the world\'s most 
populous  Bounded by the Indian Ocean on the south, the Arabian Sea on the southwest, and the Bay of Bengal on the 
southeast, it shares land borders with Pakistan to the west;[k] China, Nepal, and Bhutan to the north; and 
Bangladesh and Myanmar to the east. In the Indian Ocean, India is near Sri Lanka and the Maldives; its Andaman and 
Nicobar Islands share a maritime border with Thailand, Myanmar, and Indonesia.\n\nModern humans arrived on the 
Indian subcontinent from Africa no later than 55,000 years ago.[26][27][28] Their long occupation, predominantly in
isolation as hunter-gatherers, has made the region highly diverse.[29] Settled life emerged on the subcontinent in 
the western margins of the Indus river basin 9,000 years ago, evolving gradually into the Indus Valley Civilisation
of the third millennium BCE.[30] By 1200\xa0BCE, an archaic form of Sanskrit, an Indo-European language, had 
diffused

## 6. Using Generative Models

Initialize and use generative models like `ChatGoogleGenerativeAI` and `ChatOpenAI` for text generation tasks. The `get_generative_model` function helps you select and configure the model.

In [13]:
llmgoogle = get_generative_model(model_name='gemini-1.5-flash',
                    type='google',
                    _tools=None,
                    kwargs={'temperature': 0.1, 'max_tokens': None, 'timeout': None, 'max_retries': 2, 
                            'api_key': os.environ['GOOGLE_API_KEY'],
                            'generation_config':{"response_mime_type": "application/json"}})

# For local (ollama, for others I will add the support in coming weeks)

# llmlocal = get_generative_model(model_name='qwen:0.5b-chat',
#                     type='local',
#                     _tools=None,
#                     kwargs={'temperature': 0.1, 'max_tokens': None, 'timeout': None, 'max_retries': 2})

[2025-06-08 13:34:09,033] WARNING langchain_google_genai.chat_models: Unexpected argument 'generation_config' provided to ChatGoogleGenerativeAI.
[2025-06-08 13:34:09,033] WARNING langchain_google_genai.chat_models: Unexpected argument 'generation_config' provided to ChatGoogleGenerativeAI.
/var/folders/sh/v0nx5vbx4mv07k1h4mxmwxcw0000gn/T/ipykernel_97548/4078719958.py:1: UserWarning: WARNING! generation_config is not default parameter.
                generation_config was transferred to model_kwargs.
                Please confirm that generation_config is what you intended.
  llmgoogle = get_generative_model(model_name='gemini-1.5-flash',


In [14]:
web_response = await query_web_response(
    "Top news of today",
    '08-06-2025',
    'Friday',
    searcher,
    hf_embeddings,
    True,
    cross_encoder,
    llmgoogle,
    text_model=llmgoogle,
    num_results=1,
    document_paths=[],
    local_mode=False,
    split=False
)
print(web_response[0])

[2025-06-08 13:34:19,279] INFO utils.websearch_utils: Search phrases for query 'Top news of today': ['top news', 'latest news']
[2025-06-08 13:34:19,279] INFO utils.websearch_utils: Search phrases for query 'Top news of today': ['top news', 'latest news']
[2025-06-08 13:34:19,940] INFO utils.websearch_utils: Search results for query 'top news': [{'snippet': 'Breaking News in India: Read Latest News on Sports, Business, Entertainment, Blogs and Opinions from leading columnists. Times of India brings the Breaking ...', 'title': 'Times of India: News - Breaking News, Latest News, India ...', 'link': 'https://timesofindia.indiatimes.com/', 'engines': ['google'], 'category': 'general'}]
[2025-06-08 13:34:19,940] INFO utils.websearch_utils: Search results for query 'top news': [{'snippet': 'Breaking News in India: Read Latest News on Sports, Business, Entertainment, Blogs and Opinions from leading columnists. Times of India brings the Breaking ...', 'title': 'Times of India: News - Breaking 

Summarising using 2 documents

[2025-06-08 13:34:30,045] INFO utils.websearch_utils: Response generated for query 'Top news of today'.
[2025-06-08 13:34:30,045] INFO utils.websearch_utils: Response generated for query 'Top news of today'.


Top news stories from the Times of India for today include:

* **International:**  The ongoing feud between Elon Musk and Donald Trump continues to make headlines, with reports
of a physical altercation between Musk and a banker, and Trump's reactions.  Russia's retaliation against Ukraine 
following drone attacks on Moscow is also major news.  A Colombian presidential candidate was shot during a rally. 
Israel discovered weapons and control rooms in a tunnel under a Gaza hospital.  The recapture of a fugitive killer 
and ex-police chief in Arkansas is reported.

* **National (India):**  A large-scale fraud at ICICI Bank involving a former employee is under investigation.  The
death of a man in a stampede, and the grief of his father, is highlighted.  Congress criticized PM Modi for not 
holding a press conference in 11 years.  Virat Kohli is the last active member of India's 2011 World Cup-winning 
squad.  A man in Bengaluru beheaded his wife.  A spiritual and wellness summit is to be held in Ujjain.

* **Business:**  The stock market's performance next week will be influenced by global trends and inflation data.  
A significant fraud at ICICI Bank is detailed.

* **Technology:** Apple is set to unveil iOS 26 with major redesigns and AI enhancements at WWDC 2025.  The death 
of Bill Atkinson, an engineer who helped create the Apple Lisa and Macintosh computers, is reported.

* **Sports:** Team India's training visuals in England have been shared by the BCCI.  Coco Gauff won the French 
Open title.

* **Other:**  Several other stories cover various topics including health (liver disease symptoms, Ozempic side 
effects), entertainment (Bollywood and Hollywood news), and lifestyle.


The articles provide details on each of these stories, including quotes from individuals involved and analysis of 
the events.  The specific details within each category are numerous and varied.

## 9. YouTube Transcript Summarization

Summarize YouTube video transcripts using the `youtube_transcript_response` function. This is useful for extracting insights from long videos.

In [15]:
summary = youtube_transcript_response("https://www.youtube.com/watch?v=o8NiE3XMPrM&t=6648s", 
                                      'summarize in bullets and themes', llmgoogle)
print(summary)

Google I/O 2024 Keynote Summary: A Deep Dive into Gemini & Beyond 🚀

This Google I/O keynote showcased a massive leap forward in Google's AI capabilities, primarily driven by the 
advancements in their Gemini model.  The presentation focused on Gemini's rapid progress, its integration into 
various Google products, and the broader implications for AI's future.  Here's a summary by themes and bullet 
points:


**I. Gemini's Stunning Progress & Performance 🏆**

* **Unprecedented Speed & Efficiency:**  Gemini's Elo scores have increased by over 300 points since its first 
generation, and Gemini 2.5 Pro now dominates leaderboards across coding (WebDev Arena #1, 140 Elo points 
improvement) and large language model benchmarks (LMArena, all categories).  It's generating hundreds of thousands 
of lines of accepted code *per minute* on Cursor! 🤯
* **Multimodal Mastery:** Gemini isn't just text-based; it's multimodal, excelling at tasks involving images, 
video, and audio.  A highlight was Gemini completing Pokémon Blue – a testament to its advanced reasoning and 
problem-solving abilities.  This achievement is playfully referred to as "API" (Artificial Pokémon Intelligence). 
😂
* **Underlying Infrastructure:** Google's seventh-generation TPU, Ironwood, is a key enabler, delivering a 10x 
performance boost over its predecessor and boasting 42.5 exaflops of compute per pod. This powerful infrastructure 
allows for faster model development and deployment at lower costs. ⚡️


**II. Gemini's Integration into Google Products & Services 📱💻🔍**

* **Google Search:** AI Overviews now have over 1.5 billion monthly users, demonstrating massive adoption.  The new
AI Mode in Search offers a completely reimagined search experience, handling complex, multi-step queries and 
providing rich, dynamic responses (including graphs and maps).  It leverages Gemini's advanced reasoning and query 
fanout techniques for deeper, faster results.  Personal context will further personalize results based on user data
(Gmail, etc.). 🔎
* **Google Gemini App:**  Boasting over 400 million monthly active users, the Gemini app is getting a major upgrade
with Agent Mode (allowing multi-step task completion), Deep Research (for in-depth analysis), and personalized 
Smart Replies in Gmail.  The app also integrates with Google services like Calendar, Maps, and Keep.  Agent Mode 
uses Project Mariner to automate tasks like apartment hunting. 🏡
* **Google Meet:** Real-time speech translation is now available in Google Meet (English and Spanish initially, 
more languages coming soon), making cross-language communication seamless. 🗣️
* **Android & XR:** Gemini is deeply integrated into Android, accessible via the power button and coming soon to 
wearables and other devices.  Android XR, a new platform for headsets and glasses, leverages Gemini to provide 
context-aware assistance, navigation, and immersive experiences.  Partnerships with Samsung, Gentle Monster, and 
Warby Parker are bringing Android XR glasses to market. 👓


**III.  Frontier Research & Future Directions 🔮**

* **Project Astro:** This universal AI assistant prototype is now integrated into Gemini Live, allowing users to 
share their camera view and discuss what they see.  It's surprisingly good at correcting user misconceptions! 🤣
* **Project Mariner:**  This agent can interact with the web and perform multiple tasks simultaneously (up to 10). 
Its "Teach and Repeat" feature allows it to learn from user interactions and improve its performance.  It's being 
integrated into the Gemini API and Chrome. 🌐
* **Gemini Deep Think:** This new mode pushes Gemini 2.5 Pro's performance to its limits, achieving impressive 
results on challenging benchmarks (USAMO, LiveCodeBench, MMMU).  It's currently undergoing further safety 
evaluations before wider release. 🤔
* **Gemini Diffusion:** A new text diffusion model that generates text five times faster than previous models while
maintaining coding performance.  This parallel g

## 10. Generating Maps

Generate maps with routes and points of interest using the `generate_map` function. You can visualize directions and locations directly in your notebook.

In [16]:
# Example: Generate a map with a route and POIs
from utils.map import *
s = generate_map("M G Road, Bangalore", "Indiranagar, Bangalore")
from IPython.display import display, HTML
with open("output/map_with_route_and_pois.html") as f:
    html_content = f.read()
# display(HTML(html_content))

[2025-06-08 13:34:48,028] INFO utils.map: Found 3 probable locations for 'M G Road, Bangalore'.
[2025-06-08 13:34:48,029] INFO utils.map: Auto-selected location: Mahatma Gandhi Road, Tasker Town, Shivajinagar, Bengaluru, Bangalore North, Bengaluru Urban, Karnataka, 560001, India
[2025-06-08 13:34:49,020] INFO utils.map: Found 3 probable locations for 'Indiranagar, Bangalore'.
[2025-06-08 13:34:49,022] INFO utils.map: Auto-selected location: Indiranagar, Basaveshwaranagar, Bengaluru, Bangalore North, Bengaluru Urban, Karnataka, 560079, India
[2025-06-08 13:34:49,366] INFO utils.map: Route found between start and end coordinates.
[2025-06-08 13:34:49,996] INFO utils.map: Found 78 POIs near (12.9747828, 77.6096698).
[2025-06-08 13:34:50,393] INFO utils.map: Found 12 POIs near (12.9962979, 77.5452778).
[2025-06-08 13:34:50,470] INFO utils.map: Map generated and saved as 'map_with_route_and_pois.html'.
[2025-06-08 13:34:50,470] INFO utils.map: Generated route directions.


## 11. Advanced Query Handling

Handle advanced queries, such as generating detailed reports or toy examples, using `query_web_response`. This enables complex, multi-step reasoning and content generation.

In [17]:
detailed_report = await query_web_response(
    "Give me end to end working for text diffusion model.",
    '06-06-2025',
    'Friday',
    searcher,
    hf_embeddings,
    True,
    cross_encoder,
    llmgoogle,
    text_model=llmgoogle,
    num_results=3,
    document_paths=[],
    local_mode=False,
    split=False
)
print(detailed_report[0])

[2025-06-08 13:34:51,852] INFO utils.websearch_utils: Search phrases for query 'Give me end to end working for text diffusion model.': ['text diffusion architecture', 'train text diffusion', 'run text diffusion', 'text diffusion use', 'text diffusion limits', 'future text diffusion']
[2025-06-08 13:34:51,852] INFO utils.websearch_utils: Search phrases for query 'Give me end to end working for text diffusion model.': ['text diffusion architecture', 'train text diffusion', 'run text diffusion', 'text diffusion use', 'text diffusion limits', 'future text diffusion']
[2025-06-08 13:34:52,596] INFO utils.websearch_utils: Search results for query 'text diffusion architecture': [{'snippet': "22 May 2025 — What's special about diffusion models that makes text generation so much faster? Should every text model be a diffusion model, going forward?", 'title': 'Strengths and limitations of diffusion language models', 'link': 'https://www.seangoedecke.com/limitations-of-text-diffusion-models/', 'en

Fetching posts from 
https://www.reddit.com/r/StableDiffusion/comments/wl4cn3/the_maximum_usable_length_of_a_stable_diffusion/

[2025-06-08 13:34:58,454] INFO utils.websearch_utils: Encoding end for URL: https://arxiv.org/abs/2306.00637, Time taken: 0.44 seconds
[2025-06-08 13:34:58,454] INFO utils.websearch_utils: Encoding end for URL: https://arxiv.org/abs/2306.00637, Time taken: 0.44 seconds
[2025-06-08 13:34:58,456] INFO utils.websearch_utils: Reranking start for URL: https://arxiv.org/abs/2306.00637
[2025-06-08 13:34:58,456] INFO utils.websearch_utils: Reranking start for URL: https://arxiv.org/abs/2306.00637
[2025-06-08 13:34:58,456] INFO utils.websearch_utils: Reranking end for URL: https://arxiv.org/abs/2306.00637, Time taken: 0.00 seconds
[2025-06-08 13:34:58,456] INFO utils.websearch_utils: Reranking end for URL: https://arxiv.org/abs/2306.00637, Time taken: 0.00 seconds


Fetching comments for post: The maximum usable length of a Stable Diffusion text prompt is purportedly 77 tokens. 
Here is what that means, and how to test how many tokens are in your text prompt.

[2025-06-08 13:34:58,717] INFO utils.websearch_utils: Processed markdown for: https://www.seangoedecke.com/limitations-of-text-diffusion-models/
[2025-06-08 13:34:58,717] INFO utils.websearch_utils: Processed markdown for: https://www.seangoedecke.com/limitations-of-text-diffusion-models/
[2025-06-08 13:34:58,719] INFO utils.websearch_utils: Successfully processed and added document(s) for URL: https://www.seangoedecke.com/limitations-of-text-diffusion-models/
[2025-06-08 13:34:58,719] INFO utils.websearch_utils: Successfully processed and added document(s) for URL: https://www.seangoedecke.com/limitations-of-text-diffusion-models/
[2025-06-08 13:34:58,842] INFO utils.websearch_utils: Processed markdown for: https://huggingface.co/docs/diffusers/en/tutorials/basic_training
[2025-06-08 13:34:58,842] INFO utils.websearch_utils: Processed markdown for: https://huggingface.co/docs/diffusers/en/tutorials/basic_training
[2025-06-08 13:34:58,843] INFO utils.websearch_utils: Successfully proce

Fetching posts from 
https://www.reddit.com/r/LocalLLaMA/comments/1jmgj6z/is_there_a_future_for_diffusion_language_models/

Batches: 100%|██████████| 1/1 [00:00<00:00, 20.95it/s]


Fetching comments for post: is there a future for diffusion language models ?

[2025-06-08 13:35:10,793] INFO utils.websearch_utils: Retrieved 1 docs for https://www.reddit.com/r/StableDiffusion/comments/wl4cn3/the_maximum_usable_length_of_a_stable_diffusion/
[2025-06-08 13:35:10,793] INFO utils.websearch_utils: Retrieved 1 docs for https://www.reddit.com/r/StableDiffusion/comments/wl4cn3/the_maximum_usable_length_of_a_stable_diffusion/
[2025-06-08 13:35:10,794] INFO utils.websearch_utils: Built context for https://www.reddit.com/r/StableDiffusion/comments/wl4cn3/the_maximum_usable_length_of_a_stable_diffusion/
[2025-06-08 13:35:10,794] INFO utils.websearch_utils: Built context for https://www.reddit.com/r/StableDiffusion/comments/wl4cn3/the_maximum_usable_length_of_a_stable_diffusion/
[2025-06-08 13:35:10,795] INFO utils.websearch_utils: Successfully processed and added context/docs for URL: https://www.reddit.com/r/StableDiffusion/comments/wl4cn3/the_maximum_usable_length_of_a_stable_diffusion/
[2025-06-08 13:35:10,795] INFO utils.websearch_utils: Successfully 

#### Answer: 
Text diffusion models offer a different approach to text generation compared to autoregressive models.  They 
generate the entire output at once, potentially leading to faster generation. However, they have limitations, such 
as slower processing of long contexts and challenges in complex reasoning tasks.  The end-to-end workflow involves 
training a model (often a transformer-based network) on a large text dataset to learn to denoise noisy text 
sequences.  Inference involves starting with a noisy sequence and iteratively denoising it until a coherent text is
produced.  Specific implementations and training details vary across different models.

#### Next steps:
More research into specific text diffusion model architectures (e.g., Masked Diffusion Models) and their training 
processes is needed to provide a complete answer.  The available search results offer partial information on 
various aspects, including training, inference, and limitations.

In [18]:
# You can even summarise full page 
detailed_report = await query_web_response(
    "Summarise this full page in detail, and give me learning notes https://ollama.com/blog/secureminions",
    '06-06-2025',
    'Friday',
    searcher,
    hf_embeddings,
    True,
    cross_encoder,
    llmgoogle,
    text_model=llmgoogle,
    num_results=3,
    document_paths=[],
    local_mode=False,
    split=False
)
print(detailed_report[0][0])

[2025-06-08 13:35:30,662] INFO utils.websearch_utils: Search phrases for query 'Summarise this full page in detail, and give me learning notes https://ollama.com/blog/secureminions': ['summarize ollama blog post', 'secure minions learning notes']
[2025-06-08 13:35:30,662] INFO utils.websearch_utils: Search phrases for query 'Summarise this full page in detail, and give me learning notes https://ollama.com/blog/secureminions': ['summarize ollama blog post', 'secure minions learning notes']
[2025-06-08 13:35:30,663] INFO utils.websearch_utils: Extracted URLs from query 'Summarise this full page in detail, and give me learning notes https://ollama.com/blog/secureminions': ['https://ollama.com/blog/secureminions']
[2025-06-08 13:35:30,663] INFO utils.websearch_utils: Extracted URLs from query 'Summarise this full page in detail, and give me learning notes https://ollama.com/blog/secureminions': ['https://ollama.com/blog/secureminions']
[2025-06-08 13:35:30,664] INFO utils.websearch_utils: 

## Secure Minions: Detailed Summary and Learning Notes

This Ollama blog post details Secure Minions, a secure, private collaboration protocol between local LLMs (like 
Ollama's models) and powerful cloud-based frontier models (like GPT-4o).  It builds upon the original Minions 
project from Stanford's Hazy Research lab, significantly enhancing privacy.

**Original Minions (Context):** The original Minions project aimed to reduce cloud costs and improve efficiency by 
processing most of the work locally.  While it kept raw context local, some information still went to the cloud, 
posing a privacy risk.

**Secure Minions (Innovation):** Secure Minions addresses this privacy concern by implementing end-to-end 
encryption, even from the cloud provider. This is achieved using NVIDIA's Hopper H100 GPUs' confidential computing 
mode.  No plaintext is exposed during transmission or remote LLM inference.  Even with large prompts (around 8k 
tokens) and models like Qwen-32B, the added latency is minimal (less than 1%).

**Key Features and Improvements:**

* **Enhanced Privacy:**  End-to-end encryption prevents sensitive data from being accessible to the cloud provider.
* **Cost-Effective:**  Maintains the cost savings of the original Minions (5x-30x reduction in cloud costs compared
to using only frontier models).
* **High Accuracy:** Achieves approximately 98% of the accuracy of using a frontier model alone.
* **Minimal Overhead:**  The added security layer introduces negligible latency.

**Technical Implementation:**

Secure Minions leverages the confidential computing capabilities of NVIDIA's Hopper H100 GPUs.  The blog post 
refers to a more detailed technical explanation on the HazyResearch blog.  The provided code examples demonstrate 
how to use the `minions` Python package to interact with both the original and secure protocols.  The examples show
how to set up a local Ollama client, a secure remote client, and how to send a task and context to the system.

**Getting Started:**

The blog post provides clear instructions on how to:

1. **Clone the repository:** `git clone ...`
2. **Create a virtual environment (optional):** `python3 -m venv .venv` and `source .venv/bin/activate`
3. **Install the package:** `pip install -e .`
4. **Install Ollama and a model (e.g., Gemma 3):** `ollama pull gemma3:4b`
5. **Run the Streamlit app (interactive demo):** `streamlit run app.py`  This allows users to experiment with both 
the secure and insecure protocols.
6. **Run example code programmatically:**  The blog provides a Python example demonstrating the use of the 
`SecureClient` and `OllamaClient` classes.


**Learning Notes:**

* **Confidential Computing:**  This technology is crucial for Secure Minions.  Further research into confidential 
computing and its applications in AI is recommended.
* **Local-First Architectures:**  The benefits of local-first approaches for privacy and cost-effectiveness are 
highlighted.
* **LLM Orchestration:**  Secure Minions demonstrates a sophisticated approach to orchestrating local and remote 
LLMs for optimal performance and privacy.
* **Python Programming:**  The provided code examples offer practical experience in using Python for LLM 
interaction.
* **Streamlit:**  The use of Streamlit for creating interactive demos is a valuable learning point.


This blog post showcases a significant advancement in secure and private LLM collaboration.  The combination of 
local processing, efficient cloud usage, and robust encryption makes Secure Minions a compelling solution for 
privacy-sensitive applications.

In [19]:
# or multiple pages
detailed_report = await query_web_response(
    "Summarise https://ollama.com/blog/secureminions and https://github.com/ggml-org/llama.cpp",
    '06-06-2025',
    'Friday',
    searcher,
    hf_embeddings,
    True,
    cross_encoder,
    llmgoogle,
    text_model=llmgoogle,
    num_results=3,
    document_paths=[],
    local_mode=False,
    split=False
)
print(detailed_report[0])

[2025-06-08 13:35:43,462] INFO utils.websearch_utils: Search phrases for query 'Summarise https://ollama.com/blog/secureminions and https://github.com/ggml-org/llama.cpp': ['summarize secureminions', 'summarize llama.cpp']
[2025-06-08 13:35:43,462] INFO utils.websearch_utils: Search phrases for query 'Summarise https://ollama.com/blog/secureminions and https://github.com/ggml-org/llama.cpp': ['summarize secureminions', 'summarize llama.cpp']
[2025-06-08 13:35:43,462] INFO utils.websearch_utils: Extracted URLs from query 'Summarise https://ollama.com/blog/secureminions and https://github.com/ggml-org/llama.cpp': ['https://ollama.com/blog/secureminions', 'https://github.com/ggml-org/llama.cpp']
[2025-06-08 13:35:43,462] INFO utils.websearch_utils: Extracted URLs from query 'Summarise https://ollama.com/blog/secureminions and https://github.com/ggml-org/llama.cpp': ['https://ollama.com/blog/secureminions', 'https://github.com/ggml-org/llama.cpp']
[2025-06-08 13:35:43,463] INFO utils.webse

Summarising using 2 documents

[2025-06-08 13:35:53,498] INFO utils.websearch_utils: Response generated for query 'Summarise https://ollama.com/blog/secureminions and https://github.com/ggml-org/llama.cpp'.
[2025-06-08 13:35:53,498] INFO utils.websearch_utils: Response generated for query 'Summarise https://ollama.com/blog/secureminions and https://github.com/ggml-org/llama.cpp'.


Here's a summary of the provided documents addressing the user's request to summarize both the Ollama Secure 
Minions blog post and the llama.cpp GitHub repository.

**Ollama Secure Minions (https://ollama.com/blog/secureminions):**

This blog post describes Secure Minions, a secure, private collaboration protocol built upon the original Minions 
project from Stanford's Hazy Research lab.  Minions allows local LLMs (like Ollama's models) to collaborate with 
powerful cloud-based frontier models (like GPT-4o) while minimizing data sent to the cloud, thus reducing costs and
improving privacy.  Secure Minions enhances this by adding end-to-end encryption, even from the cloud provider, 
using NVIDIA's Hopper H100 GPUs' confidential computing mode.  This ensures that no plaintext is exposed during 
transmission or inference, with minimal latency overhead (less than 1%). The blog post provides instructions for 
setting up and running Secure Minions, including using a Streamlit app and example Python code.  The core benefit 
is significantly improved privacy by keeping sensitive data local while still leveraging the power of larger cloud 
models.


**llama.cpp (https://github.com/ggml-org/llama.cpp):**

This GitHub repository contains a C/C++ implementation for running large language models (LLMs).  It focuses on 
efficient inference across various hardware platforms (CPU, GPU, etc.), aiming for minimal setup and high 
performance.  The project supports several LLMs, primarily focusing on Meta's LLaMA models and their finetuned 
variants.  Users can download models (often in GGUF format) from Hugging Face or other sources.  The repository 
includes various tools: `llama-cli` (a command-line interface), `llama-server` (an OpenAI API-compatible server), 
`llama-bench` (for benchmarking and measuring model quality), and example applications (`llama-simple`).  The 
project also provides documentation on adding support for new models and quantization techniques to optimize 
performance.  Pre-compiled versions (XCFrameworks) are available for iOS, macOS, and other Apple platforms.

In [20]:
# you can even do search/summary within local files

# or multiple pages
detailed_report = await query_web_response(
    "Summarise this document",
    '06-06-2025',
    'Friday',
    searcher,
    hf_embeddings,
    True,
    cross_encoder,
    llmgoogle,
    text_model=llmgoogle,
    num_results=3,
    document_paths=[['documents/1706.03762v7.pdf']],
    local_mode=True,
    split=False
)
print(detailed_report[0])

[2025-06-08 13:35:54,390] INFO utils.websearch_utils: Search phrases for query 'Summarise this document': ['summarize document']
[2025-06-08 13:35:54,390] INFO utils.websearch_utils: Search phrases for query 'Summarise this document': ['summarize document']
[2025-06-08 13:35:54,391] WARNING utils.websearch_utils: No websearcher provided; using document_paths only.
[2025-06-08 13:35:54,391] WARNING utils.websearch_utils: No websearcher provided; using document_paths only.
[2025-06-08 13:35:54,392] INFO utils.websearch_utils: Starting context_to_docs for 1 URL groups.
[2025-06-08 13:35:54,392] INFO utils.websearch_utils: Starting context_to_docs for 1 URL groups.
/opt/homebrew/Cellar/python@3.13/3.13.1/Frameworks/Python.framework/Versions/3.13/lib/python3.13/multiprocessing/resource_tracker.py:136: UserWarning: resource_tracker: process died unexpectedly, relaunching.  Some resources might leak.
  warnings.warn('resource_tracker: process died unexpectedly, '
[2025-06-08 13:35:54,404] INF

Summarising using 2 documents

[2025-06-08 13:36:02,544] INFO utils.websearch_utils: Response generated for query 'Summarise this document'.
[2025-06-08 13:36:02,544] INFO utils.websearch_utils: Response generated for query 'Summarise this document'.


This paper introduces the Transformer, a novel neural network architecture for sequence transduction tasks like 
machine translation.  Unlike dominant models that rely on recurrent or convolutional neural networks, the 
Transformer uses only attention mechanisms, eliminating recurrence and convolutions entirely.  This allows for 
significantly greater parallelization during training, resulting in faster training times and improved translation 
quality.

The Transformer achieves state-of-the-art results on the WMT 2014 English-to-German and English-to-French 
translation tasks, surpassing previous best results (including ensembles) by a significant margin, while requiring 
substantially less training time.  The model also generalizes well to other tasks, demonstrating strong performance
on English constituency parsing with both large and limited training data.

The core innovation is the use of "scaled dot-product attention" and "multi-head attention," which allow the model 
to efficiently capture long-range dependencies between input and output sequences.  The paper details the 
architecture, including encoder and decoder stacks, attention mechanisms, positional encoding, and training 
procedures.  Experiments demonstrate the effectiveness of various components of the Transformer architecture, such 
as the number of attention heads and the impact of regularization techniques like dropout and label smoothing.  The
authors provide visualizations of the attention mechanism, showcasing its ability to capture syntactic and semantic
relationships within sentences.  The code used for training and evaluation is publicly available.

The next cell demonstrates how to summarize Reddit posts and comments from the "OpenAI" subreddit using the `reddit_reader_response` function. It retrieves the top 2 "hot" posts and summarizes their content with the help of the `llmgoogle` generative model. This is useful for quickly extracting key insights from Reddit discussions.

In [21]:
from utils.reddit_utils import *
summary = reddit_reader_response(
  subreddit="OpenAI",
  url_type="hot",
  n=2,
  k=2,
  custom_url=None,
  time_filter="today",
  search_query=None,
  sort_type="hot",
  model=llmgoogle
)
print(summary)

Fetching posts from /r/OpenAI

Fetching comments for post: Sooo... OpenAI is saving all ChatGPT logs "indefinitely"... Even deleted ones...

Fetching comments for post: Lawsuit must be won. This is absurd

The Reddit posts discuss OpenAI's indefinite storage of ChatGPT logs, sparked by a lawsuit.

The first post, titled "Sooo... OpenAI is saving all ChatGPT logs 'indefinitely'... Even deleted ones..." 
((https://www.reddit.com/r/OpenAI/comments/1l5z78y/sooo_openai_is_saving_all_chatgpt_logs/)), lacks original text 
but its comments reveal the indefinite storage is likely a court order to preserve evidence for a legal case.  One 
commenter suggests this is for discovery purposes and the "indefinitely" clause is a simplification, subject to the
judge's rulings on the case's duration. Another commenter mentions the involvement of ICE (Immigration and Customs 
Enforcement), implying potential legal ramifications related to the content of the chat logs.

The second post, "Lawsuit must be won. This is absurd" 
((https://www.reddit.com/r/OpenAI/comments/1l64rqh/lawsuit_must_be_won_this_is_absurd/)), argues against the 
indefinite storage of chat logs. The author contends that mandating such storage for one company is ineffective 
against crime, as criminals would simply switch platforms.  They highlight the disproportionate impact on the 
privacy of innocent users and the immense costs and environmental damage associated with such a policy.  The author
believes that if permanent storage is deemed necessary, it should be applied universally, but acknowledges the 
impracticality and absurdity of such a measure.

The comments on the second post offer further context. One commenter points out the potential violation of EU law. 
Another commenter suggests the lawsuit is not primarily about crime, but rather the *New York Times*' copyright 
infringement lawsuit against OpenAI.  This commenter theorizes that the lawsuit aims to damage ChatGPT's user base 
and that both a win or loss for the NYT could have negative long-term consequences.  A win could open the 
floodgates for copyright lawsuits against US AI companies, giving a competitive advantage to those outside the US, 
similar to repealing Section 230. A loss would solidify fair use for AI training data.

---

This concludes the tutorial for the **coexist**. You have learned how to perform web search, document processing, answer generation, YouTube summarization, and more. For further details, refer to the project README or explore the codebase for advanced usage patterns. 